<a href="https://colab.research.google.com/github/justinhtn/scratch-tf/blob/main/digit_predictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Our Goal
Build a model that can predict the labels of any set of handwritten digits between 0-9 that's thrown its way. 👍

### **Recomended reading:**
- [Keras Docs](https://keras.io/)
- [Sequential model guide](https://keras.io/getting-started/sequential-model-guide/)
- [Keras Github](https://github.com/keras-team/keras/)

In [2]:
# Sequential is a Keras model in which each layer is stacked on top of each other.
from keras.models import Sequential

# The following imports will be used for creating individual network layers.
# 'Conv2, MaxPooling2D, Flatten and Dropout' will be used to specifically accomodate the
# convolutional layers while 'Dense' will allow us to create fully connected layers.
# We'll go through these in more detail when we use them later in the notebook.
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Adding early stopping will stop our network when its performance improvement starts to dwindle.
from keras.callbacks import EarlyStopping

# We'll be working with the MNIST dataset which consists of 60,000 28x28
# images along with a test set of 10,000 images. Both training and test sets include labels.
from keras.datasets import mnist

# Importing extra utils from keras, numPy and scikit learn to plot and transform training and test data
from keras.utils import np_utils
from keras.utils.vis_utils import plot_model
from tensorflow.keras.utils import to_categorical
from keras.preprocessing.image import load_img, img_to_array

import numpy as np
from numpy import argmax

import matplotlib.pyplot as plt

from skimage.transform import resize

# loading saved models back into our notebook
from keras.models import load_model

## Load data
We'll set up a load_data function to do the following:
- Load training and testing data from the MNIST dataset into a tuple.
- Reshape the data into a shape that CNN's require. [Batch size, Width, Height, Number of Channels]
- [One Hot Encode](https://hackernoon.com/what-is-one-hot-encoding-why-and-when-do-you-have-to-use-it-e3c6186d008f) the training and test lables.

In [3]:
def load_data():
# loading dataset
  (X_train, y_train), (X_test, y_test) = mnist.load_data()
 
# reshape and convery type
  X_train = X_train.reshape((X_train.shape[0], 28, 28, 1))
  X_test = X_test.reshape((X_test.shape[0], 28, 28, 1))
 
# one hot encoding labels
  y_train = to_categorical(y_train)
  y_test = to_categorical(y_test)
  
  return X_train, y_train, X_test, y_test

## Transform data
Next, we'll create a prep_data function that will pre-process that loaded data:
- Take in the training and test image data that the load_data() function returns.
- Convert training and test data to float32 format.
- Normalize data to values between 0-1 rather than 0-255.

In [4]:
def prep_data(train_imgs, test_imgs):
# convert to float
  train_imgs_float = train_imgs.astype('float32')
  test_imgs_float = test_imgs.astype('float32')
 
# noralize between 0-1
  train_imgs_normal = train_imgs_float / 255.0
  test_imgs_normal = test_imgs_float / 255.0

  return train_imgs_normal, test_imgs_normal

## Model building
The sequential model we will be building will consist of 11 layers because it's fun to experiment. We will utilize the keras layers Conv2D, Flatten, Dense, Dropout and MaxPooling2D.

**Resources:**
[Keras core layers](https://keras.io/layers/core/)

In [5]:
def cnn_model():
  model = Sequential()
  model.add(Conv2D(50, (3, 3), activation='relu', input_shape=(28, 28, 1)))
  model.add(Conv2D(50, (3, 3), activation='relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(0.25))

  model.add(Conv2D(50, (3, 3), activation='relu'))
  model.add(Conv2D(50, (3, 3), activation='relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(0.25))
  
  model.add(Flatten())
  model.add(Dense(150, activation='relu'))
  model.add(Dense(10, activation='softmax'))

# compiling and save model
  model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

  return model

In [6]:
cnn_instance = cnn_model()
cnn_instance.summary()

## Training and Evaluating the model

In [7]:
def train_and_evaluate():
  model = cnn_model()
  X_train, y_train, X_test, y_test = load_data()
  X_train, X_test = prep_data(X_train, X_test)
  early_stopping = EarlyStopping(patience=3)
  model.fit(X_train, y_train, epochs=5, batch_size=150, callbacks=[early_stopping])
  model.save("cnn_v4.h5")

  test_accuracy = model.evaluate(X_test, y_test)
  train_accuracy = model.evaluate(X_train, y_train)

  print(f"Test loss: {round(test_accuracy[0]*100,5)}%")
  print(f"Test accuracy: {round(test_accuracy[1]*100, 5)}%")
  print(f"Error percentage: {round(100-test_accuracy[1]*100,5)}%")

  print(f"Train loss: {round(train_accuracy[0]*100, 5)}%")
  print(f"Train accuracy: {round(train_accuracy[1]*100, 5)}%")
  print(f"Error percentage: {round(100-train_accuracy[1]*100,5)}%")

In [8]:
train_and_evaluate()

11501568/11490434 [==============================] - 0s 0us/step
Epoch 1/5
400/400 [==============================] - 19s 24ms/step - loss: 0.2267 - accuracy: 0.9303
Epoch 2/5
400/400 [==============================] - 9s 23ms/step - loss: 0.0576 - accuracy: 0.9818
Epoch 3/5
400/400 [==============================] - 9s 23ms/step - loss: 0.0429 - accuracy: 0.9865
Epoch 4/5
400/400 [==============================] - 9s 24ms/step - loss: 0.0330 - accuracy: 0.9896
Epoch 5/5
1875/1875 [==============================] - 10s 5ms/step - loss: 0.0117 - accuracy: 0.9963
Test loss: 1.92527%
Test accuracy: 99.35%
Error percentage: 0.65%
Train loss: 1.16798%
Train accuracy: 99.62834%
Error percentage: 0.37166%


##Loading and Predict Single Image

In [9]:
def load_test_image(image_path):
# load in the image and convert to grey scale. 
# ** NOTE ** load_img also normalizes the image file
	img_loaded = load_img(image_path, color_mode="grayscale")
# resize image to 28x28
	img_resized = resize(np.array(img_loaded), (28, 28))
# reshape to have 1 channel and 1 for the batch size
	img_reshaped = img_resized.reshape((1, 28, 28, 1))
# convert to float
	img_normal = img_reshaped.astype('float32')
	return img_normal

In [13]:
def predictImage(image_path, model_path):
  loaded_image = load_test_image(image_path)
  model = load_model(model_path)
  digit = np.argmax(model.predict(loaded_image), axis=-1)
  print(digit)

In [14]:
predictImage('/content/34367226-1897-4D1C-AF6E-57AB7156B7DE.jpeg', 'cnn_v4.h5')

[3]


In [ ]:
!!git clone https://gist.github.com/8409b3feec20f159d8a50b0a811d3bca.git

["Cloning into '8409b3feec20f159d8a50b0a811d3bca'...",
 'remote: Enumerating objects: 6, done.\x1b[K',
 'remote: Total 6 (delta 0), reused 0 (delta 0), pack-reused 6\x1b[K',
 'Unpacking objects:  16% (1/6)   ',
 'Unpacking objects:  33% (2/6)   ',
 'Unpacking objects:  50% (3/6)   ',
 'Unpacking objects:  66% (4/6)   ',
 'Unpacking objects:  83% (5/6)   ',
 'Unpacking objects: 100% (6/6)   ',
 'Unpacking objects: 100% (6/6), done.']

In [ ]:
%run /content/8409b3feec20f159d8a50b0a811d3bca/draw.py